# 1. Loading Tags to RDS

#### Uncomment the following cell to download the data

In [9]:
from sqlalchemy import create_engine, text
import sqlalchemy as sqla
import pandas as pd
import cx_Oracle
import boto3
import warnings
import numpy as np
from Loaders import SQLLoader
warnings.filterwarnings("ignore")

# If requiered
import sqlalchemy as sa
from sqlalchemy import Float, Double
from sqlalchemy.dialects import oracle

In [10]:
ssm = boto3.client('ssm')
dbhost = ssm.get_parameter(Name='/dbtester/dbhost', WithDecryption=True)['Parameter']['Value']
dbport = ssm.get_parameter(Name='/dbtester/dbport', WithDecryption=True)['Parameter']['Value']
dbuser = ssm.get_parameter(Name='/dbtester/dbuser', WithDecryption=True)['Parameter']['Value']
dbpass = ssm.get_parameter(Name='/dbtester/dbpass', WithDecryption=True)['Parameter']['Value']
dbname = ssm.get_parameter(Name='/dbtester/dbname', WithDecryption=True)['Parameter']['Value']

# A. Creating Loaders

In [12]:
file_tags= 'app-data/tags.csv'
dbengine = 'oracle+cx_oracle'
dbtable  = 'tags'
dtype    = {"userId":np.str, "movieId":np.str, "tag":np.str, "timestamp":np.str}
dtype_db = {
    "userId":sa.FLOAT,
    "movieId":sa.FLOAT,
    "tag":sqla.types.VARCHAR(100),
    "timestamp":sa.FLOAT,
    "_insert":sa.FLOAT,
    "_insert_time":sa.FLOAT,
    "_update":sa.FLOAT,
    "_update_time":sa.FLOAT,
    "_delete":sa.FLOAT,
    "_delete_time":sa.FLOAT,
}

tloader = SQLLoader(
    file_tags,
    dbengine,
    dbhost,
    dbuser,
    dbpass,
    dbport,
    dbname,
    dbtable,
    drop=True,
    dtype=dtype,
    dtype_db=dtype_db)
display(tloader.df.head(3), tloader.status())

,userId,movieId,tag,timestamp,_insert,_insert_time,_update,_update_time,_delete,_delete_time
0,3,260,classic,1439472355,0,0,0,0,0,0
1,3,260,sci-fi,1439472256,0,0,0,0,0,0
2,4,1732,dark comedy,1573943598,0,0,0,0,0,0


{'state': 'AVAILABLE',
 'iteration': 0,
 'inserted': 0,
 'updated': 0,
 'deleted': 0,
 'indb': None}

--------

# B. SQL Loading
### B1. Full-Load

In [ ]:
# THE UID LOAD MUST CONTAIN ONLY INSERTS, THIS WAY THE TABLE CAN BE CREATED AND
# THE ENGINE EXECUTE OF THE SECOND LINE CAN ALTER THE TABLE TO ALLOW DELETS REPLICATION
tloader.iud(inserts=100, updates=0 , deletes=0, max_registers=2000)
tloader.engine.execute(f"ALTER TABLE {dbtable} REPLICA IDENTITY FULL");

### B2. CDC (Inserts, Updates, Deletes)

In [ ]:
tloader.iudx(inserts=100, updates=10, deletes=1)

#### Extra

Use 👇 to delete the table
```python
tloader.drop_table()
````